# Kunskapskontroll 2 - köra skript
Namn: Nil Abukar

In [6]:
pip install pyodbc

Note: you may need to restart the kernel to use updated packages.


In [22]:
import requests
import pyodbc

# Hämtar data från en API och retunerar den i JSON-format och loggar eventuella fel.
def fetch_data_from_api():
    try:
        response = requests.get('https://jsonplaceholder.typicode.com/todos/1')
        if response.status_code == 200:
            data = response.json()
            print('Data från API har hämtats:')
            return data
        else:
            print(f'Fel vid API-anrop: {response.status_code}')
            return None
    except Exception as e:
        log_error(f'Fel vid hämtning av data från API: {e}')
        return None

# Formaterar och validerar data    
def process_data(data):
    try:
        title = data['title'].strip().lower() 
        completed = int(data['completed'])  

        if validate_data(title, completed):
            print("Data har bearbetats!")
            return title, completed
        else:
            return None
    except Exception as e:
        log_error(f"Fel vid bearbetning av data: {e}")
        return None
    
def validate_data(title, completed):
    if not isinstance(title, str) or not title:
        log_error("Ogiltig titel!")
        return False
    if completed not in (0, 1):
        log_error("Ogiltigt värde för completed!")
        return False
    return True

# Kontrollerar om titeln redan finns i databasen och loggar ett meddelande om det finns en dubletter.
def check_duplicate(title):
    conn = create_connection()
    cursor = conn.cursor()
    cursor.execute('SELECT COUNT(*) FROM todos WHERE title = ?', (title,))
    exists = cursor.fetchone()[0] > 0
    conn.close()
    return exists

def log_error(error_message):
    with open('error_log.txt', 'a') as log_file:
        log_file.write(error_message + '\n')

# Skapar en anslutning till SQL server och skapar en tabell om den inte redan finns. En bearbetat data läggs till i databasen om det inte finns dubbletter.        
def create_connection():
    conn = pyodbc.connect(
        'Driver={SQL Server};'
        'Server=RAQI;'
        'Database=raqis_database;'
        'Trusted_Connection=yes;'
    )
    return conn

def create_table(conn):
    cursor = conn.cursor()
    cursor.execute('''
        IF NOT EXISTS (SELECT * FROM sysobjects WHERE name='todos' AND xtype='U')
        BEGIN
            CREATE TABLE todos (
                id INT IDENTITY(1,1) PRIMARY KEY,
                title NVARCHAR(255),
                completed BIT
            )
        END
    ''')
    conn.commit()

def update_database(processed_data):
    title, completed_value = processed_data
    if check_duplicate(title):
        log_error(f"Dubblett hittad för titeln: {title}")
        return  # Avbryt om dubblett finns

    conn = create_connection()
    cursor = conn.cursor()
    cursor.execute('INSERT INTO todos (title, completed) VALUES (?, ?)', (title, completed_value))
    conn.commit()
    conn.close()

if __name__ == '__main__':
    conn = create_connection()
    create_table(conn)
    conn.close()
    
    data = fetch_data_from_api()
    if data is not None:
        processed_data = process_data(data)
        if processed_data is not None:
            update_database(processed_data)


Data från API har hämtats:
Data har bearbetats!


# Automatiserade test uppdateringar

In [12]:
pip install pytest

Note: you may need to restart the kernel to use updated packages.


In [18]:
import sys
import os 
print(os.path.isfile(r'C:\Users\nilab\Documents\process_data.py'))

True


In [ ]:
# Unitest har använts för att säkerställa att funktionerna fungerar som förväntat. Dessa tester kontrollerar att datahämtningen, bearbetning, validering och databasuppdatering finns med.
import unittest
from process_data import fetch_data_from_api, process_data, update_database, create_connection, validate_data

class TestYourScript(unittest.TestCase):

    def test_fetch_data(self):
        data = fetch_data_from_api()
        self.assertIsNotNone(data)
        self.assertIn('title', data)
        self.assertIn('completed', data)

    def test_process_data(self):
        data = {'title': ' Test Title ', 'completed': True}
        processed_data = process_data(data)
        self.assertEqual(processed_data[0], 'test title')  # Kontrollera att titeln är formaterad korrekt
        self.assertEqual(processed_data[1], 1)  # Kontrollera att completed är int (1)

    def test_validate_data(self):
        self.assertTrue(validate_data('Valid Title', 1))
        self.assertFalse(validate_data('', 1))  # Ogiltig titel
        self.assertFalse(validate_data('Valid Title', 2))  # Ogiltigt värde för completed

    def test_update_database(self):
        processed_data = ('Test Title', 1)  # Datan som ska uppdateras
        update_database(processed_data)  # Anropa funktionen för att uppdatera databasen
        
        # Kontrollerar att datan har lagts till i databasen
        conn = create_connection()
        cursor = conn.cursor()
        cursor.execute('SELECT * FROM todos WHERE title = ?', (processed_data[0],))
        result = cursor.fetchone()
        conn.close()
        
        self.assertIsNotNone(result)  # Kontrollera att resultatet inte är None
        self.assertEqual(result[1], 'Test Title')  # Kontrollera att titeln är korrekt
        self.assertEqual(result[2], 1)  # Kontrollera att completed är True (1 i SQL)

if __name__ == '__main__':
    unittest.main()